In [10]:
import json
import itertools
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, BoxAnnotation

In [11]:
class Data(object):  # each data object contains a time, distance, and rssi value
    
    def __init__(self):
        self.time = 0
        self.distance = 0
        self.rssi = 0

In [12]:
# global variables
beaconNum = "0:0" 
allBeacons = {}  # dictionary of every beacon in a block
#colors = itertools.cycle(["red", "green", "yellow", "blue", "orange", "magenta", "black"])  # different colors for each beacon
#colors = itertools.cycle(["red", "blue"])  # different colors for distance and rssi
colors = itertools.cycle(["firebrick"])
x = []  # time
y = []  # distance
z = []  # rssi
average_dist = [0] * 400
average_rssi = [0] * 400
average_var = [0] * 400
begin = []
end = []
initial_time = 0

In [13]:
def moving_average(window):  # parameter represents window for average moving window filter
# first time frame: 0-2 seconds -> get average for second 1
# second time frame: 1-3 seconds -> get average for second 2
# keep incrementing by 1 second, but frame is [window] seconds
    global allBeacons
    global x  # list of all times
    global y  # list of all distances
    global z  # list of all rssi values
    global beaconNum
    global average_dist
    global average_rssi
    global average_var
    variance = []
    numInWindow = 0
    for i in range(400):  # for every single second 0 thru 399
        for j in range(len(x)): # loop through every single time 0 thru len(x)-1
            if x[j] >= i - (window/2) and x[j] < i + (window/2):  # if time is in proper frame
                numInWindow += 1
                average_dist[i] = average_dist[i] + y[j]
                average_rssi[i] = average_rssi[i] + z[j]
                variance.append(y[j])  # list of dist values within proper frame and for specific second
        if numInWindow > 0:
            average_dist[i] = average_dist[i] / numInWindow
            average_rssi[i] = average_rssi[i] / numInWindow
        for k in range(len(variance)):
            variance[k] = (variance[k] - average_dist[i])**2  # convert values to variance
            average_var[i] = average_var[i] + variance[k]
        average_var[i] = average_var[i] / len(variance)
        variance.clear()  # clear list for next second
        numInWindow = 0

In [14]:
def plot_minor():
    global allBeacons
    global beaconNum
    global colors
    global x  # times
    global y  # distances
    global z  # rssi values
    global average_dist
    global average_rssi
    global average_var
    global begin
    global end
    global initial_time
    color = next(colors)
    tempTitle="Beacon " + str(beaconNum)

    for i in range(len(allBeacons[beaconNum])):  # position in allBeacon[beaconNum] array
        x.append(allBeacons[beaconNum][i].time)
        y.append(allBeacons[beaconNum][i].distance)
        z.append(allBeacons[beaconNum][i].rssi)
    
    moving_average(4) # calculate moving average for distance and rssi in 4 second window
    
    output_notebook()
    p = figure(
        tools="pan, box_zoom, wheel_zoom, reset, resize",
        title=tempTitle, x_axis_label='Average Distance (meters)', y_axis_label='Variance',
        x_range=(min(average_dist), max(average_dist)), y_range=(min(average_var), max(average_var)),
        plot_width=900, plot_height=400
    )
    
    # plot average variances for each second
    p.scatter(
        average_dist, average_var,
        marker="circle", fill_color=color, line_color = color
    )
    
    show(p)

In [15]:
def get_data(i):  # open text file and extract data
    global beaconNum
    global allBeacons
    global initial_time
    
    textName = "minor" + str(i) + ".txt"
    f = open(textName)
    first = True
    for line in f:
        words = line.split()
        time = int(words[0])  # get time
        if first == True:
            initial_time = time  # set base time
            first = False
        time = (time - initial_time) / 1000  # get elapsed time in seconds
        parsed_json = json.loads(words[1])  # parse json
        beaconNum = str(parsed_json['major']) + ":" + str(parsed_json['minor'])  # beacon identified by major and minor
        rssi = parsed_json['rssi']
        distance = 10**((rssi+60)/-20)  # log-distance pathloss model
        allBeacons.setdefault(beaconNum, []).append(Data())  # each beacon has an array of Data
        dataPos = len(allBeacons[beaconNum]) - 1  # position in array
        allBeacons[beaconNum][dataPos].time = time
        allBeacons[beaconNum][dataPos].distance = distance
        allBeacons[beaconNum][dataPos].rssi = rssi
    f.close()  # close file

In [16]:
def get_markers():
    global begin
    global end
    m = open("markers.txt")
    for line in m:
        words = line.split()
        time = int(words[0])  # get raw time
        parsed_json = json.loads(words[1])  # parse json
        if parsed_json['marker'] == "begin\n":
            begin.append(time)
        elif parsed_json['marker'] == "end\n":
            end.append(time)
    m.close()

In [17]:
def main():
    get_markers()
    for i in range(7):
        get_data(i)
        plot_minor()

In [18]:
if __name__ == "__main__":
    main()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...